In [ ]:
import copy
import itertools
import random
from collections import namedtuple
import numpy as np

In [ ]:
class GameState():
  def __init__(self, board, utility=None, nivel=0):
    self.__board=board
    self.__utility=utility
    self.__nivel=nivel

  @property
  def board(self):
    return self.__board

  @board.setter
  def board(self, value):
    self.__board = value
  
  @property
  def utility(self):
    return self.__utility

  @utility.setter
  def utility(self, value):
    self.__utility = value
  
  @property
  def nivel(self):
    return self.__nivel

  @nivel.setter
  def nivel(self, value):
    self.__nivel = value

In [ ]:
class Game(object):
    # Identificar cuales son los movimientos o jugadas posibles de un jugador
    def actions(self, state):
        raise NotImplementedError

    # Ejecutar un movimiento o jugada y retornar el nuevo estado
    def result(self, state, move):
        raise NotImplementedError

    # Calcular la función de utilidad
    def utility(self, state):
        raise NotImplementedError

    # Condición de parada (profundidad, ganador )
    def terminal_test(self, state):
        return not self.actions(state)
        
    # Ejecutar el juego
    def play_game(self, *players):
       raise NotImplementedError
   

In [ ]:
def alpha_beta_search(state, game):
    # Functions used by alpha_beta
    def max_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state)
        v = -np.inf
        for a in game.actions(state,1):
            v = max(v, min_value(game.result(state, a,1), alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state)
        v = np.inf
        for a in game.actions(state,2):
            v = min(v, max_value(game.result(state, a,2), alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_search:
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state,1):
        newstate=game.result(state, a,1)
        #print(a,newstate.board)
        v = min_value(newstate, best_score, beta)
        if v > best_score:
            best_score = v
            best_action = a
    #print('Best Action',best_action)
    return best_action

In [ ]:
from os import listxattr
import copy
import random as rnd
class Othelo():
  def __init__(self):
    self.table = [[0,2,'B'],[0,3,'B'],[0,4,'B'],[0,2,'B'],[0,3,'B'],[0,4,'B'], 
                  [1,0,'B'],[1,1,'B'],[1,2,'B'],[1,3,'B'],[1,4,'B'],[1,5,'B'],[1,6,'N'],[1,7,'N'],
                  [2,0,'B'],[2,1,'N'],[2,2,'N'],[2,3,'B'],[2,4,'B'],[2,5,'B'],[2,6,'N'],[2,7,'B'],
                  [3,0,'B'],[3,1,'N'],[3,2,'N'],[3,3,'B'],[3,4,'N'],[3,5,'B'],[3,6,'B'],[3,7,'B'],
                  [4,0,'B'],[4,1,'B'],[4,2,'N'],[4,3,'B'],[4,4,'N'],[4,5,'N'],[4,6,'B'],[4,7,'N'],
                  [5,0,'B'],[5,1,'B'],[5,2,'N'],[5,3,'B'],[5,4,'B'],[5,5,'B'],[5,6,'B'],[5,7,'B'],
                  [6,0,'B'],[6,1,'N'],[6,2,'N'],[6,3,'B'],[6,4,'N'],[6,5,'B'],[6,6,'B'],[6,7,'N'],
                  [7,0,'B'],[7,1,'B'],[7,2,'B'],[7,3,'B'],[7,4,'B'],[7,5,'N'],[7,6,'B'],[7,7,'B']]
    self.nivel = 3
    self.tablero = []

    for i in range(8):
      self.tablero.append([' ']*8)
  
  #paso un arreglo de todas las posibles jugadas, solo la ficha donde se va a jugar
  def actions(self, state, player):
    board=state.board
    actions=[]
    color = 0
    if(player == 1):
      color = 'B'
    else:
      color = 'N'  
    lista= encontrar_adyacentes_validos(board,color)
    for x in lista:
      actions.append(x[-1])    
    return actions



  #realizo mi logica para dar solucion al ejercicio planteado
  def play_game(self, *players):
    state=GameState(self.table)
    tamanio_tablero = 0
    dibujarTablero(state.board,self.tablero)
    while tamanio_tablero <= 64:
        tamanio_tablero = len(self.table)
      #usuario   
        fila=0
        columna=0
        color='N'
        print("Juega con las fichas negras (N)")
        print("ingrese la fila: ")
        fila = int(input())
        print("ingrese la columna: ")
        columna = int(input())  
        jugada_usuario = [fila,columna,'N']
        camino = verificar_jugada_usuario(self.table,jugada_usuario)
        if camino != [] :
          self.table.append(jugada_usuario)
          pintar_tablero(self.table,camino,color)
          print("Jugada usuario")
          dibujarTablero(state.board,self.tablero)
          print("")
          print("")
        else:
          print("Jugada no valida.....")  
          print("Ingrese una jugada valida....")
      #maquina
       
        if camino != []:
          comp=alpha_beta_search(state,self)
          print("Jugada maquina ",comp)
          #En esta parte se verifica si con una jugada puede pintar varios caminos,
          #entonces se procede a pintar los caminos.
          union_camino=revisar(self.table,'B',comp)
          if union_camino != [0]:
            self.table.append(comp)    
            print("Jugada maquina")
            pintar_tablero(self.table,union_camino,'B')
            dibujarTablero(state.board,self.tablero)
          #si no se cumple la condicion se procede a pintar solo un camino   
          else:
             lista= encontrar_adyacentes_validos(state.board,'B') 
             for x in lista:
              if x[-1]==comp:
                camino = x
                self.table.append(comp)    
                print("Jugada maquina")
                pintar_tablero(self.table,camino,'B')
                dibujarTablero(state.board,self.tablero)
                print("")
                print("")
          camino=[]
        
    board=state.board
    banclas=0
    negras=0
    diferecia=0  
    for y in board:
     if y[-1]=='B':
      banclas=banclas+1
     else:
      negras=negras+1  

    if banclas > negras:    
      print("El ganador es el de las fichas de color Blancas (B)")   
    else:  
      print("El ganador es el de las fichas de color Negras (N)")   
  
  #segun la accion pinto el tablero
  def result(self, state, action, player):    
    board=state.board
    color = 0
    if(player == 1):
      color = 'B'
    else:
      color = 'N' 
    newboard=copy.deepcopy(board)
    lista= encontrar_adyacentes_validos(board,color)
    for x in lista:
      if x[-1]==action:
        camino = x
    pintar_tablero(newboard,camino,color)    
    newboard.append(action)
    n=state.nivel+1
    newstate=GameState(newboard, nivel=n)
    return newstate
  
  #En esta utilidad se realiza la diferencia del numero de posibles jugadas
  #que tienes las fichas de la maquina(Blancas) y el numero de posibles jugadas
  #de las fichas del usuario(Negras).
  def utility(self, state):
    board=state.board
    num_posibles_jugadas_banclas=0
    num_posibles_jugadas_negras=0
    diferencia = 0
    num_posibles_jugadas_banclas = len(encontrar_adyacentes_validos(board,'B'))
    num_posibles_jugadas_negras = len(encontrar_adyacentes_validos(board,'N'))
    diferencia = num_posibles_jugadas_banclas - num_posibles_jugadas_negras
    return diferencia
 
  
  def terminal_test(self, state):
    
    if state.nivel==self.nivel:
      return True
    else:
      return False

In [ ]:
from posixpath import commonpath
from re import A
import copy


def pintar_tablero(tablero,camino,color):
  for x in tablero:
    for y in camino:
      if x == y:
        x[2]=color
 
 
#llega el tablero con las fichas que estan y el color con el que debe jugar la maquina.
def encontrar_adyacentes_validos(tablero,color):
  if(color=='B'):
    aux_color1 = 'B'
    aux_color2 = 'N'
  else:
    aux_color1 = 'N'
    aux_color2 = 'B' 
  
  aux = copy.deepcopy(tablero)
  caminos_abajo=[]
  caminos_arriba=[]
  caminos_izquierda=[]
  caminos_derecha=[]
  caminos_izquierda_arriba=[]
  caminos_derecha_arriba=[]
  caminos_izquierda_abajo=[]
  caminos_derecha_abajo=[]  
  for x in range(len(aux)):
    if (tablero[x][2] == aux_color1):
      ficha=copy.deepcopy(tablero[x])
     
     
      lista=adyacencia_abajo(tablero,ficha,aux_color1,aux_color2)
      if(lista != None and lista != []):
        caminos_abajo.append(lista)
     
      lista=adyacencia_arriba(tablero,ficha,aux_color1,aux_color2)
      if(lista != None and lista !=[]):
        caminos_arriba.append(lista)  

      lista=adyacencia_izquierda(tablero,ficha,aux_color1,aux_color2) 
      if(lista != None and lista!=[]):
        caminos_izquierda.append(lista) 

      lista=adyacencia_derecha(tablero,ficha,aux_color1,aux_color2)
      if(lista != None and lista!=[]):
        caminos_derecha.append(lista)  

      lista=adyacencia_iqz_arr(tablero,ficha,aux_color1,aux_color2)
      if(lista != None and lista!=[]):
        caminos_izquierda_arriba.append(lista) 

      lista=adyacencia_der_arr(tablero,ficha,aux_color1,aux_color2)
      if(lista != None and lista!=[]):
        caminos_derecha_arriba.append(lista) 

      lista=adyacencia_izq_abj(tablero,ficha,aux_color1,aux_color2)
      if(lista != None and lista!=[]):
        caminos_izquierda_abajo.append(lista)  

      lista=adyacencia_der_abj(tablero,ficha,aux_color1,aux_color2)
      if(lista != None and lista!=[]):
        caminos_derecha_abajo.append(lista)
 
  lista_caminos=[]
  lista_caminos.extend(caminos_arriba)
  lista_caminos.extend(caminos_izquierda)
  lista_caminos.extend(caminos_derecha)
  lista_caminos.extend(caminos_abajo)
  lista_caminos.extend(caminos_izquierda_arriba)
  lista_caminos.extend(caminos_derecha_arriba)
  lista_caminos.extend(caminos_izquierda_abajo)
  lista_caminos.extend(caminos_derecha_abajo)

  aux_lis_camino=copy.deepcopy(lista_caminos)

  return lista_caminos

def adyacencia_abajo(tablero,ficha,aux_color1,aux_color2):
 aux=[]
 aux2=[]
 aux_ficha = copy.deepcopy(ficha)
 aux_ficha[0]=aux_ficha[0]+1
 aux_ficha[2]=aux_color2
 while(tablero.count(aux_ficha)):
  aux3=copy.deepcopy(aux_ficha)
  aux2.append(aux3)
  aux_ficha[0]=aux_ficha[0]+1
 #Como el auxiliar ficha queda con una posición adelante cuando ya no se cumpla la condicion del ciclo, entonces esa posición
 #nos sirve como la jugada ya que no encuentra mas fichas contrarias para ir guardando el camino.
 jugada=copy.deepcopy(aux_ficha) 
 jugada[2]=aux_color1
 #Se verifica que la jugada no sea invalida es decir que si encuentra una ficha del mismo color() bajando ya no sera jugada valida
 #o que la ficha este fuera del rango del tablero.
 aux = copy.deepcopy(aux_ficha)
 aux[2]=aux_color1
 aux_ficha[0]=aux_ficha[0]-1
 if ((tablero.count(jugada)) or jugada[0]>7):
   return []
 elif aux2!=[]:
   aux2.append(jugada)
   return aux2     

def adyacencia_arriba(tablero,ficha,aux_color1,aux_color2):
 aux=[]
 aux2=[]
 aux_ficha = copy.deepcopy(ficha)
 aux_ficha[0]=aux_ficha[0]-1
 aux_ficha[2]=aux_color2
 while(tablero.count(aux_ficha)):
  aux3=copy.deepcopy(aux_ficha)
  aux2.append(aux3)
  aux_ficha[0]=aux_ficha[0]-1
 jugada=copy.deepcopy(aux_ficha) 
 jugada[2]=aux_color1
 #Se verifica que la jugada no sea invalida es decir que si encuentra una ficha del mismo color() subiendo ya no sera jugada valida
 #o que la ficha este fuera del rango del tablero.
 aux = copy.deepcopy(aux_ficha)
 aux[2]=aux_color1
 aux_ficha[0]=aux_ficha[0]+1
 if ((tablero.count(jugada)) or jugada[0]<0 ):
   return []
 elif aux2!=[]:
   aux2.append(jugada)
   return aux2     


def adyacencia_izquierda(tablero,ficha,aux_color1,aux_color2):
 aux=[]
 aux2=[]
 aux_ficha = copy.deepcopy(ficha)
 aux_ficha[1]=aux_ficha[1]-1
 aux_ficha[2]=aux_color2
 while(tablero.count(aux_ficha)):
  aux3=copy.deepcopy(aux_ficha)
  aux2.append(aux3)
  aux_ficha[1]=aux_ficha[1]-1
 jugada=copy.deepcopy(aux_ficha) 
 jugada[2]=aux_color1 
  
 #Se verifica que la jugada no sea invalida es decir que si encuentra una ficha del mismo color() izquierda ya no sera jugada valida
 #o que la ficha este fuera del rango del tablero.
 aux = copy.deepcopy(aux_ficha)
 aux[2]=aux_color1
 aux_ficha[1]=aux_ficha[1]+1
 if ((tablero.count(jugada))or jugada[1]<0):
   return []
 elif aux2!=[]:
   aux2.append(jugada)
   return aux2   
 

def adyacencia_derecha(tablero,ficha,aux_color1,aux_color2):
 aux=[]
 aux2=[]
 aux_ficha = copy.deepcopy(ficha)
 aux_ficha[1]=aux_ficha[1]+1
 aux_ficha[2]=aux_color2
 while(tablero.count(aux_ficha)):
  aux3=copy.deepcopy(aux_ficha)
  aux2.append(aux3)
  aux_ficha[1]=aux_ficha[1]+1
 jugada=copy.deepcopy(aux_ficha) 
 jugada[2]=aux_color1 
 #Se verifica que la jugada no sea invalida es decir que si encuentra una ficha del mismo color() derecha ya no sera jugada valida
 #o que la ficha este fuera del rango del tablero.
 aux = copy.deepcopy(aux_ficha)
 aux[2]=aux_color1
 aux_ficha[1]=aux_ficha[1]-1
 if ((tablero.count(jugada))or jugada[1]>7):
   return []
 elif aux2!=[]:
   aux2.append(jugada)
   return aux2  

def adyacencia_iqz_arr(tablero,ficha,aux_color1,aux_color2):
 aux=[]
 aux2=[]
 aux_ficha = copy.deepcopy(ficha)
 aux_ficha[0]=aux_ficha[0]-1
 aux_ficha[1]=aux_ficha[1]-1
 aux_ficha[2]=aux_color2
 while(tablero.count(aux_ficha)):
  aux3=copy.deepcopy(aux_ficha)
  aux2.append(aux3)
  aux_ficha[0]=aux_ficha[0]-1
  aux_ficha[1]=aux_ficha[1]-1
 jugada=copy.deepcopy(aux_ficha) 
 jugada[2]=aux_color1 
 #Se verifica que la jugada no sea invalida es decir que si encuentra una ficha del mismo color() izquierda arriba ya no sera jugada valida
 #o que la ficha este fuera del rango del tablero.
 aux = copy.deepcopy(aux_ficha)
 aux[2]=aux_color1
 aux_ficha[0]=aux_ficha[0]+1
 aux_ficha[1]=aux_ficha[1]+1
 if ((tablero.count(jugada))or(jugada[0]<0 or jugada[1]<0)):
   return []
 elif aux2!=[]:
   aux2.append(jugada)
   return aux2  

def adyacencia_der_arr(tablero,ficha,aux_color1,aux_color2):
 aux=[]
 aux2=[]
 aux_ficha = copy.deepcopy(ficha)
 aux_ficha[0]=aux_ficha[0]-1
 aux_ficha[1]=aux_ficha[1]+1
 aux_ficha[2]=aux_color2
 while(tablero.count(aux_ficha)):
  aux3=copy.deepcopy(aux_ficha)
  aux2.append(aux3)
  aux_ficha[0]=aux_ficha[0]-1
  aux_ficha[1]=aux_ficha[1]+1
 jugada=copy.deepcopy(aux_ficha) 
 jugada[2]=aux_color1 
 #Se verifica que la jugada no sea invalida es decir que si encuentra una ficha del mismo color() derecha arriba ya no sera jugada valida
 #o que la ficha este fuera del rango del tablero.
 aux = copy.deepcopy(aux_ficha)
 aux[2]=aux_color1
 aux_ficha[0]=aux_ficha[0]+1
 aux_ficha[1]=aux_ficha[1]-1
 if ((tablero.count(jugada))or (jugada[0]<0 and jugada[1]>7)):
   return []
 elif aux2!=[]:
   aux2.append(jugada)
   return aux2  
  
def adyacencia_izq_abj(tablero,ficha,aux_color1,aux_color2):
 aux=[]
 aux2=[]
 aux_ficha = copy.deepcopy(ficha)
 aux_ficha[0]=aux_ficha[0]+1
 aux_ficha[1]=aux_ficha[1]-1
 aux_ficha[2]=aux_color2
 while(tablero.count(aux_ficha)):
  aux3=copy.deepcopy(aux_ficha)
  aux2.append(aux3)
  aux_ficha[0]=aux_ficha[0]+1
  aux_ficha[1]=aux_ficha[1]-1
 jugada=copy.deepcopy(aux_ficha) 
 jugada[2]=aux_color1 
 #Se verifica que la jugada no sea invalida es decir que si encuentra una ficha del mismo color() izquierda abajo ya no sera jugada valida
 #o que la ficha este fuera del rango del tablero.
 aux = copy.deepcopy(aux_ficha)
 aux[2]=aux_color1
 aux_ficha[0]=aux_ficha[0]-1
 aux_ficha[1]=aux_ficha[1]+1
 if ((tablero.count(jugada))or(jugada[0]>7 and jugada[1]<0)):
   return []
 elif aux2!=[]:
   aux2.append(jugada)
   return aux2  
   
def adyacencia_der_abj(tablero,ficha,aux_color1,aux_color2):
 aux=[]
 aux2=[]
 aux_ficha = copy.deepcopy(ficha)
 aux_ficha[0]=aux_ficha[0]+1
 aux_ficha[1]=aux_ficha[1]+1
 aux_ficha[2]=aux_color2
 while(tablero.count(aux_ficha)):
  aux3=copy.deepcopy(aux_ficha)
  aux2.append(aux3)
  aux_ficha[0]=aux_ficha[0]+1
  aux_ficha[1]=aux_ficha[1]+1
 jugada=copy.deepcopy(aux_ficha) 
 jugada[2]=aux_color1 
 #Se verifica que la jugada no sea invalida es decir que si encuentra una ficha del mismo color() derecha abajo ya no sera jugada valida
 #o que la ficha este fuera del rango del tablero.
 aux = copy.deepcopy(aux_ficha)
 aux[2]=aux_color1
 aux_ficha[0]=aux_ficha[0]-1
 aux_ficha[1]=aux_ficha[1]-1
 if ((tablero.count(jugada))or (jugada[0]>7 and jugada[1]>7)):
   return []
 elif aux2!=[]:
   aux2.append(jugada)
   return aux2  

In [ ]:
#othelo=Othelo()
#state=GameState([[3,3,'B'],[4,3,'N'],[3,4,'N'],[4,4,'B'],[4,2,'N']])
#comp=alpha_beta_search(state,othelo)
#print(comp)

def dibujarTablero(borad,tablero):
    for i in borad:
        x=i[0]
        y=i[1]
        z=i[2]
        tablero[x][y]=z  
    print("===================================")
    print("  | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |")
    print("===================================")
    numero = 0
    for i in tablero:
        print(numero,end=' | ')
        for j in range(len(tablero)):
          print(i[j], end=' | ')
        print('\n')
        numero +=1
       

In [ ]:
#Esta funcion se encarga de verificar si la jugada ingresada por el usuario 
#es correcta, si es correcta retorna el camino de lo contrario retorna la lista vacia, cabe recalcal que esta funcion utiliza la funcion
#revisar, para verificar si la jugada puede pintar multiples caminos.
def verificar_jugada_usuario(tablero,jugada_usuario):
  posibles_jugadas = revisar(tablero,'N',jugada_usuario)
  aux_posi_jugadas= encontrar_adyacentes_validos(tablero,'N')
  if posibles_jugadas[-1]== jugada_usuario:
    camino = posibles_jugadas
    return camino
  for x in aux_posi_jugadas:
    if x[-1] == jugada_usuario:
      return x  
  return [] 
  
    

In [ ]:
#Esta funcion se encarga de verificar si una jugada puede pintar varios caminos, y si es asi
#esta funcion concatena los dos caminos para asi pintarlos 
def revisar(tablero,color,jugada):
  aux_camino=[]
  jugadas=[]
  aux_jugada=0
  contador=0
  posibles_jugadas = encontrar_adyacentes_validos(tablero,color)
  for x in posibles_jugadas:
    if x!= [] or x!=None:
      jugadas.append(x[-1]) 
  aux= posibles_jugadas[-1]
  contador = jugadas.count(jugada)
  for x in posibles_jugadas:  
    if contador > 1 and x[-1]==jugada:
      aux_jugada = x.pop()
      aux_camino.extend(x)
  aux_camino.append(aux_jugada) 
  return aux_camino


In [ ]:
def main():
    juego=Othelo()
    juego.play_game();

if __name__ == "__main__":
      main()

  | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
0 |   |   | B | B | B |   |   |   | 

1 | B | B | B | B | B | B | N | N | 

2 | B | N | N | B | B | B | N | B | 

3 | B | N | N | B | N | B | B | B | 

4 | B | B | N | B | N | N | B | N | 

5 | B | B | N | B | B | B | B | B | 

6 | B | N | N | B | N | B | B | N | 

7 | B | B | B | B | B | N | B | B | 

Juega con las fichas negras (N)
ingrese la fila: 
0
ingrese la columna: 
5
Jugada usuario
  | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
0 |   |   | B | B | B | N |   |   | 

1 | B | B | B | B | N | N | N | N | 

2 | B | N | N | N | B | N | N | B | 

3 | B | N | N | B | N | N | B | B | 

4 | B | B | N | B | N | N | B | N | 

5 | B | B | N | B | B | B | B | B | 

6 | B | N | N | B | N | B | B | N | 

7 | B | B | B | B | B | N | B | B | 



Jugada maquina  [0, 7, 'B']
Jugada maquina
  | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
0 |   |   | B | B | B | N |   | B | 

1 | B | B | B | B | N | N | B | B | 

2 | B | N | N | N | B | B | N | B | 

3 | B | N | N | B | B | N | B | B |